Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [78]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from math import sqrt

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [7]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [8]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [32]:
batch_size = 128
beta = 0.001
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + beta * tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [34]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 23.182602
Minibatch accuracy: 10.2%
Validation accuracy: 10.2%
Minibatch loss at step 500: 2.721687
Minibatch accuracy: 76.6%
Validation accuracy: 77.2%
Minibatch loss at step 1000: 1.558756
Minibatch accuracy: 83.6%
Validation accuracy: 79.1%
Minibatch loss at step 1500: 1.177129
Minibatch accuracy: 82.0%
Validation accuracy: 80.4%
Minibatch loss at step 2000: 0.984190
Minibatch accuracy: 80.5%
Validation accuracy: 81.9%
Minibatch loss at step 2500: 0.762792
Minibatch accuracy: 81.2%
Validation accuracy: 82.1%
Minibatch loss at step 3000: 1.021904
Minibatch accuracy: 77.3%
Validation accuracy: 82.2%
Test accuracy: 88.7%


In [48]:
batch_size = 128
hidden1_units = 1024
beta = 5e-4
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden1_units]))
  biases1 = tf.Variable(tf.zeros([hidden1_units]))

  weights2 = tf.Variable(
    tf.truncated_normal([hidden1_units, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  
  # Training computation.
  logits = tf.matmul(hidden1, weights2) + biases2
  loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) 
  regularization = beta * tf.nn.l2_loss(weights1) + beta * tf.nn.l2_loss(biases1) +\
    beta * tf.nn.l2_loss(weights2) + beta * tf.nn.l2_loss(biases2)
  loss += regularization

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [50]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 477.066772
Minibatch accuracy: 9.4%
Validation accuracy: 26.4%
Minibatch loss at step 500: 132.301498
Minibatch accuracy: 82.0%
Validation accuracy: 79.6%
Minibatch loss at step 1000: 96.207100
Minibatch accuracy: 88.3%
Validation accuracy: 80.1%
Minibatch loss at step 1500: 75.563431
Minibatch accuracy: 83.6%
Validation accuracy: 83.0%
Minibatch loss at step 2000: 58.210724
Minibatch accuracy: 82.0%
Validation accuracy: 82.8%
Minibatch loss at step 2500: 44.661282
Minibatch accuracy: 89.8%
Validation accuracy: 83.3%
Minibatch loss at step 3000: 34.750526
Minibatch accuracy: 81.2%
Validation accuracy: 82.8%
Minibatch loss at step 3500: 26.639164
Minibatch accuracy: 93.0%
Validation accuracy: 85.7%
Minibatch loss at step 4000: 21.372078
Minibatch accuracy: 84.4%
Validation accuracy: 86.3%
Minibatch loss at step 4500: 16.247900
Minibatch accuracy: 92.2%
Validation accuracy: 86.8%
Minibatch loss at step 5000: 12.983526
Minibatch accuracy: 89.1%
Valida

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [53]:
num_steps = 3001
restricted_train_dataset = train_dataset[0:batch_size*3,:]
restricted_train_labels = train_labels[0:batch_size*3,:]
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (restricted_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = restricted_train_dataset[offset:(offset + batch_size), :]
    batch_labels = restricted_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 540.023010
Minibatch accuracy: 10.2%
Validation accuracy: 27.4%
Minibatch loss at step 500: 122.656982
Minibatch accuracy: 100.0%
Validation accuracy: 73.2%
Minibatch loss at step 1000: 95.522209
Minibatch accuracy: 100.0%
Validation accuracy: 73.2%
Minibatch loss at step 1500: 74.390526
Minibatch accuracy: 100.0%
Validation accuracy: 73.2%
Minibatch loss at step 2000: 57.933563
Minibatch accuracy: 100.0%
Validation accuracy: 73.2%
Minibatch loss at step 2500: 45.117294
Minibatch accuracy: 100.0%
Validation accuracy: 73.3%
Minibatch loss at step 3000: 35.136349
Minibatch accuracy: 100.0%
Validation accuracy: 73.3%
Test accuracy: 79.8%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [55]:
batch_size = 128
hidden1_units = 1024
beta = 5e-4
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden1_units]))
  biases1 = tf.Variable(tf.zeros([hidden1_units]))

  weights2 = tf.Variable(
    tf.truncated_normal([hidden1_units, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  keep_prob = tf.constant(0.5)
  hidden1_dropout = tf.nn.dropout(hidden1, keep_prob)
  
  # Training computation.
  logits = tf.matmul(hidden1_dropout, weights2) + biases2
  loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) 
  #regularization = beta * tf.nn.l2_loss(weights1) + beta * tf.nn.l2_loss(biases1) +\
  #  beta * tf.nn.l2_loss(weights2) + beta * tf.nn.l2_loss(biases2)
  #loss += regularization

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [54]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 570.069336
Minibatch accuracy: 7.0%
Validation accuracy: 35.9%
Minibatch loss at step 500: 131.349503
Minibatch accuracy: 81.2%
Validation accuracy: 76.3%
Minibatch loss at step 1000: 96.365768
Minibatch accuracy: 84.4%
Validation accuracy: 81.6%
Minibatch loss at step 1500: 76.672890
Minibatch accuracy: 80.5%
Validation accuracy: 82.1%
Minibatch loss at step 2000: 58.313171
Minibatch accuracy: 78.9%
Validation accuracy: 82.6%
Minibatch loss at step 2500: 44.704407
Minibatch accuracy: 88.3%
Validation accuracy: 83.7%
Minibatch loss at step 3000: 35.134808
Minibatch accuracy: 84.4%
Validation accuracy: 83.8%
Test accuracy: 90.3%


In [57]:
num_steps = 3001
restricted_train_dataset = train_dataset[0:batch_size*3,:]
restricted_train_labels = train_labels[0:batch_size*3,:]
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (restricted_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = restricted_train_dataset[offset:(offset + batch_size), :]
    batch_labels = restricted_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 552.392822
Minibatch accuracy: 10.2%
Validation accuracy: 33.0%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 76.3%
Minibatch loss at step 1000: 0.000005
Minibatch accuracy: 100.0%
Validation accuracy: 76.3%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 76.8%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 76.5%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 76.6%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 76.3%
Test accuracy: 83.1%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [68]:
batch_size = 128
hidden1_units = 1024
beta = 0.001
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden1_units]))
  biases1 = tf.Variable(tf.zeros([hidden1_units]))

  weights2 = tf.Variable(
    tf.truncated_normal([hidden1_units, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  keep_prob = tf.placeholder("float")
  hidden1_dropout = tf.nn.dropout(hidden1, keep_prob)
  
  # Training computation.
  logits = tf.matmul(hidden1_dropout, weights2) + biases2
  loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) 
  regularization = beta * tf.nn.l2_loss(weights1) + beta * tf.nn.l2_loss(biases1) +\
    beta * tf.nn.l2_loss(weights2) + beta * tf.nn.l2_loss(biases2)
  loss += regularization

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [69]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 792.818970
Minibatch accuracy: 7.8%
Validation accuracy: 28.1%
Minibatch loss at step 500: 212.547394
Minibatch accuracy: 75.0%
Validation accuracy: 78.7%
Minibatch loss at step 1000: 118.552063
Minibatch accuracy: 75.8%
Validation accuracy: 81.1%
Minibatch loss at step 1500: 71.863358
Minibatch accuracy: 79.7%
Validation accuracy: 82.8%
Minibatch loss at step 2000: 43.693279
Minibatch accuracy: 70.3%
Validation accuracy: 83.7%
Minibatch loss at step 2500: 26.020933
Minibatch accuracy: 78.9%
Validation accuracy: 84.8%
Minibatch loss at step 3000: 16.286757
Minibatch accuracy: 78.9%
Validation accuracy: 85.2%
Test accuracy: 90.9%


In [103]:
batch_size = 128
hidden1_units = 1024
hidden2_units = 300
hidden3_units = 70
beta = 0.001
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden1_units], stddev=sqrt(2.0/(image_size * image_size))))
  biases1 = tf.Variable(tf.zeros([hidden1_units]))

  weights2 = tf.Variable(
    tf.truncated_normal([hidden1_units, hidden2_units], stddev=sqrt(2.0/hidden1_units)))
  biases2 = tf.Variable(tf.zeros([hidden2_units]))
    
  weights3 = tf.Variable(
    tf.truncated_normal([hidden2_units, hidden3_units], stddev=sqrt(2.0/hidden2_units)))
  biases3 = tf.Variable(tf.zeros([hidden3_units]))
    
  weights4 = tf.Variable(
    tf.truncated_normal([hidden3_units, num_labels], stddev=sqrt(2.0/hidden3_units)))
  biases4 = tf.Variable(tf.zeros([num_labels]))
    
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  hidden2 = tf.nn.relu(tf.matmul(hidden1, weights2) + biases2)
  hidden3 = tf.nn.relu(tf.matmul(hidden2, weights3) + biases3)

  # Training computation.
  logits = tf.matmul(hidden3, weights4) + biases4
  loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) 
  regularization = beta * tf.nn.l2_loss(weights1) + beta * tf.nn.l2_loss(biases1) + \
    beta * tf.nn.l2_loss(weights2) + beta * tf.nn.l2_loss(biases2) + \
    beta * tf.nn.l2_loss(weights3) + beta * tf.nn.l2_loss(biases3) + \
    beta * tf.nn.l2_loss(weights4) + beta * tf.nn.l2_loss(biases4)

  loss += regularization

  # Optimizer.
  global_step = tf.Variable(0, trainable=False)  # count the number of steps taken.
  decay_steps = 500
  decay_rate = 0.8
  starter_learning_rate = 0.4
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, decay_steps, decay_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  hidden1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  hidden2_valid = tf.nn.relu(tf.matmul(hidden1_valid, weights2) + biases2)
  hidden3_valid = tf.nn.relu(tf.matmul(hidden2_valid, weights3) + biases3)
  valid_prediction = tf.nn.softmax(tf.matmul(hidden3_valid, weights4) + biases4)
  hidden1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  hidden2_test = tf.nn.relu(tf.matmul(hidden1_test, weights2) + biases2)
  hidden3_test = tf.nn.relu(tf.matmul(hidden2_test, weights3) + biases3)
  test_prediction = tf.nn.softmax(tf.matmul(hidden3_test, weights4) + biases4)


In [104]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.457962
Minibatch accuracy: 10.9%
Validation accuracy: 25.7%
Minibatch loss at step 500: 1.223269
Minibatch accuracy: 86.7%
Validation accuracy: 86.3%
Minibatch loss at step 1000: 0.932270
Minibatch accuracy: 90.6%
Validation accuracy: 88.0%
Minibatch loss at step 1500: 0.835424
Minibatch accuracy: 89.8%
Validation accuracy: 88.8%
Minibatch loss at step 2000: 0.914623
Minibatch accuracy: 85.2%
Validation accuracy: 89.5%
Minibatch loss at step 2500: 0.735001
Minibatch accuracy: 89.8%
Validation accuracy: 89.6%
Minibatch loss at step 3000: 0.721749
Minibatch accuracy: 88.3%
Validation accuracy: 89.6%
Minibatch loss at step 3500: 0.561384
Minibatch accuracy: 93.0%
Validation accuracy: 90.2%
Minibatch loss at step 4000: 0.717278
Minibatch accuracy: 89.1%
Validation accuracy: 90.4%
Minibatch loss at step 4500: 0.544762
Minibatch accuracy: 93.0%
Validation accuracy: 90.6%
Minibatch loss at step 5000: 0.646047
Minibatch accuracy: 92.2%
Validation accurac